In [1]:
from enum import Enum
from preprocessing_methods import *
from apify_scrap import *
from datetime import datetime
import mlflow
import pickle
from model_functions import *
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
C:\Users\michi\AppData\Roaming\Python\Python38\site-packages\visions\backends\shared\nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


In [3]:
class ImmoWeltUrls(Enum):
    BUY_FLATS_WUE_10km = "https://www.immowelt.de/liste/wuerzburg/wohnungen/kaufen?d=true&r=10&sd=DESC&sf=RELEVANCE&sp=1"
    # add price range to avoid "consulting"-offers without named price
    BUY_HOUSES_WUE_10km = "https://www.immowelt.de/liste/wuerzburg/haeuser/kaufen?d=true&pma=10000000&pmi=10&r=10&sd=DESC&sf=RELEVANCE&sp=1"
    RENT_FLATS_WUE_10km = "https://www.immowelt.de/liste/wuerzburg/wohnungen/mieten?d=true&r=10&sd=DESC&sf=RELEVANCE&sp=1"
    RENT_HOUSES_WUE_10km = "https://www.immowelt.de/liste/wuerzburg/haeuser/mieten?d=true&r=10&sd=DESC&sf=RELEVANCE&sp=1"


def getFeatureSetApp():
    return [
        "Object_price",
        "LivingSpace",
        "ZipCode",
        "Rooms",
        "altbau_(bis_1945)",
        "balkon",
        "barriefrei",
        "dachgeschoss",
        "einbaukueche",
        "neubau",
        "parkett",
        "stellplatz",
        "bad/wc_getrennt",
        "personenaufzug",
        "garten",
        "garage",
        "renoviert",
        "terrasse",
        "wanne",
        "zentralheizung",
        "abstellraum",
        "ferne",
        "fussbodenheizung",
        "gartennutzung",
        "kelleranteil",
    ]

# Dataset  issues

In [ ]:
def evaluate_model(model, X_train_recent, y_train_recent, X_val, y_val, X_test, y_test):
    pred_train = model.predict(X_train_recent)
    preds = model.predict(X_val)
    pred_test = model.predict(X_test)

    mae_train = mean_absolute_error(y_train_recent, pred_train)
    mse_train = mean_squared_error(y_train_recent, pred_train)
    r2_train = r2_score(y_train_recent, pred_train)

    mae_test = mean_absolute_error(y_test, pred_test)
    mse_test = mean_squared_error(y_test, pred_test)
    r2_test = r2_score(y_test, pred_test)

    mae_val = mean_absolute_error(y_val, preds)
    mse_val = mean_squared_error(y_val, preds)
    r2_val = r2_score(y_val, preds)

    mlflow.log_metric("mae_test", mae_test)
    mlflow.log_metric("mse_test", mse_test)
    mlflow.log_metric("r2_test", r2_test)
    mlflow.log_metric("mae_train", mae_train)
    mlflow.log_metric("r2_train", mae_train)
    mlflow.log_metric("mse_train", mae_train)
    mlflow.log_metric("mae", mae_val)
    mlflow.log_metric("mse", mse_val)
    mlflow.log_metric("r2", r2_val)
    return (
        mae_val,
        mse_val,
        r2_val,
        mae_test,
        mse_test,
        r2_test,
        mae_train,
        mse_train,
        r2_train,
    )

# Pipeline


In [ ]:
def decode_col_names(df):
    df.columns = [
        re.sub(r"\\u([0-9a-fA-F]{4})", lambda m: chr(int(m.group(1), 16)), col)
        for col in df.columns
    ]
    df.columns = [
        col.replace("ö", "oe").replace("ä", "ae").replace("ü", "ue").replace("ß", "ss")
        for col in df.columns
    ]
    return df

In [70]:
def trigger_retraining_with_added_data(
    url,
    feature_set,
    limit=3,
    model_list=["baseline-rent", "xgb", "ridge", "rf", "elasticnet", "linear", "lasso"]
):
    print(url)
    retrain_data = get_dataset_items(url, limit)
    print("Retraining data successfully scraped.")
    write_data_to_excel(retrain_data, "data/retrain_train_data.xlsx")
    print("Retraining data successfully written to excel.")

    new_df = pd.read_excel(r"data/retrain_train_data.xlsx")
    new_df = preprocess_data(new_df)
    print("Done with raw preprocessing.")
    new_df.to_excel("data/retrain_train_data_preprocessed.xlsx", index=False)

    X_val = pd.read_excel("data/X_val.xlsx")
    X_val = X_val.drop("Unnamed: 0", axis=1)
    y_val = pd.read_excel("data/y_val.xlsx")
    y_val = y_val.drop("Unnamed: 0", axis=1)
    X_test = pd.read_excel("data/X_test.xlsx")
    X_test = X_test.drop("Unnamed: 0", axis=1)
    y_test = pd.read_excel("data/y_test.xlsx")
    y_test = y_test.drop("Unnamed: 0", axis=1)

    new_df = pd.read_excel("data/retrain_train_data_preprocessed.xlsx")
    new_df = decode_col_names(new_df)

    for feature in feature_set:
        if feature not in new_df.columns:
            new_df[feature] = 0

    train_recent = pd.read_excel("data/train_recent.xlsx")
    print("old shape of train_recent", train_recent.shape)

    new_df = preprocess_data_for_model(new_df, feature_set)
    train_recent = pd.concat([train_recent, new_df], axis=0)
    train_recent = train_recent.drop_duplicates()
    print("new shape of train_recent", train_recent.shape)
    print("Retraining data successfully added to training data.")
    train_recent = train_recent.fillna(0)
    train_recent.to_excel("data/train_recent_add.xlsx", index=False)

    print("train_recent shape before dropping na", train_recent.shape)
    print("train_recent shape after dropping na", train_recent.shape)
    y_train_recent = train_recent["Object_price"]
    X_train_recent = train_recent.drop(["Object_price"], axis=1)

    now = datetime.now()
    print(
        "!!!--------------------------------------START RETRAINING----------------------------------------------!!!"
    )

    model = None
    mlflow.set_experiment(f"retraining_{now.strftime('%Y-%m-%d_%H-%M')}")

    results = pd.DataFrame()
    for model_name in model_list:
        if model_name == "xgb":
            mlflow.xgboost.autolog()
        else:
            mlflow.sklearn.autolog()

        with mlflow.start_run(run_name=f"{model_name}"):
            if model_name == "xgb":
                print("XGB------")
                print(f"train{X_train_recent.shape}")
                print(f"val:{X_val.shape}")
                print(f"y_train:{y_train_recent.shape}")
                print(f"y_val:{y_val.shape}")
                model = train_and_eval_xgb(X_train_recent, y_train_recent, X_val, y_val)
            elif model_name == "lasso":
                print("LASSO------")
                model = train_and_eval_lasso(
                    X_train_recent, y_train_recent, X_val, y_val
                )
            elif model_name == "ridge":
                print("RIDGE------")
                model = train_and_eval_ridge(
                    X_train_recent, y_train_recent, X_val, y_val
                )
            elif model_name == "rf":
                print("RF------")
                model = train_and_eval_rf(X_train_recent, y_train_recent, X_val, y_val)
            elif model_name == "elasticnet":
                print("ELASTICNET------")
                model = train_and_eval_elasticnet(
                    X_train_recent, y_train_recent, X_val, y_val
                )
            elif model_name == "linear":
                print("LINEAR------")
                model = train_and_eval_linear(
                    X_train_recent, y_train_recent, X_val, y_val
                )
            elif model_name == "baseline-rent":
                print("BASELINE-RENT------")
                avg_price = baseline_rent("", "")
                baseline_preds_val = X_val["LivingSpace"] * avg_price
                baseline_preds_test = X_test["LivingSpace"] * avg_price
                baseline_mae = mean_absolute_error(y_val, baseline_preds_val)
                baseline_r2 = r2_score(y_val, baseline_preds_val)
                baseline_mse = mean_squared_error(y_val, baseline_preds_val)
                baseline_mae_test = mean_absolute_error(y_test, baseline_preds_test)
                baseline_r2_test = r2_score(y_test, baseline_preds_test)
                baseline_mse_test = mean_squared_error(y_test, baseline_preds_test)
                print(f"Baseline Mae: {baseline_mae}")
                mlflow.log_metric("mse", baseline_mse)
                mlflow.log_metric("mae", baseline_mae)
                mlflow.log_metric("r2", baseline_r2)
                mlflow.log_metric("mse_test", baseline_mse_test)
                mlflow.log_metric("mae_test", baseline_mae_test)
                mlflow.log_metric("r2_test", baseline_r2_test)

                print(f"Baseline Mae: {baseline_mae}")
                print(f"Baseline MSE: {baseline_mse}")
                print(f"Baseline R2 Score: {baseline_r2}")

                results = results.append(
                    {
                        "model": model_name,
                        "mae": baseline_mae,
                        "mse": baseline_mse,
                        "r2": baseline_r2,
                        "mae_test": baseline_mae_test,
                        "mse_test": baseline_mse_test,
                        "r2_test": baseline_r2_test,
                    },
                    ignore_index=True,
                )

            else:
                print("Model not found.")

            print(f"Training {model_name} model done...")
            print(f"---EVALUATION AND LOGGING TO MLFLOW------ {model_name}")

            if model_name != "baseline-rent":
                (
                    mae_val,
                    mse_val,
                    r2_val,
                    mae_test,
                    mse_test,
                    r2_test,
                    mae_train,
                    mse_train,
                    r2_train,
                ) = evaluate_model(
                    model, X_train_recent, y_train_recent, X_val, y_val, X_test, y_test
                )
                results = results.append(
                    {
                        "model": model_name,
                        "mae": mae_val,
                        "mse": mse_val,
                        "r2": r2_val,
                        "mae_test": mae_test,
                        "mse_test": mse_test,
                        "r2_test": r2_test,
                        "mae_train": mae_train,
                        "mse_train": mse_train,
                        "r2_train": r2_train,
                    },
                    ignore_index=True,
                )

            mlflow.end_run()
    return results


trigger_retraining_with_added_data(
    url="https://www.immowelt.de/liste/wuerzburg/wohnungen/mieten?d=true&r=10&sd=DESC&sf=RELEVANCE&sp=1",
    feature_set=getFeatureSetApp(),
)

old shape of train_recent (108, 47)
Index(['Object_price', 'LivingSpace', 'Rooms', 'ConstructionYear', 'ZipCode',
       'EstateType', 'DistributionType', 'abstellraum', 'altbau_(bis_1945)',
       'balkon', 'barriefrei', 'dachgeschoss', 'dsl', 'dusche', 'einbaukueche',
       'elektro', 'erdgeschoss', 'etagenheizung', 'fenster', 'fern', 'ferne',
       'fliesen', 'frei', 'fussbodenheizung', 'gaestewc', 'garten',
       'gartennutzung', 'gas', 'haustiere_erlaubt', 'kelleranteil',
       'kunststoff', 'kunststofffenster', 'luftwp', 'neubau', 'offene_kueche',
       'parkett', 'pellet', 'personenaufzug', 'reinigung', 'renoviert',
       'rollstuhlgerecht', 'sat', 'speisekammer', 'stellplatz', 'terrasse',
       'tiefgarage', 'wanne', 'wg_geeignet', 'zentralheizung',
       'bad/wc_getrennt', 'garage'],
      dtype='object')
Used feature set for preprocessing:['Object_price', 'LivingSpace', 'ZipCode', 'Rooms', 'altbau_(bis_1945)', 'balkon', 'barriefrei', 'dachgeschoss', 'einbaukueche', 'n

2023/07/15 02:43:29 INFO mlflow.tracking.fluent: Experiment with name 'retraining_2023-07-15_02-43' does not exist. Creating a new experiment.


BASELINE-RENT------
Extrcated rental price per square meter via scraper: 11.21
Average rental price per sqm: 11.21
Baseline Mae: 285.5564
Baseline Mae: 285.5564
Baseline MSE: 140135.41872988813
Baseline R2 Score: 0.5802888162379594
Training baseline-rent model done...
---EVALUATION AND LOGGING TO MLFLOW------ baseline-rent


The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


XGB------
train(123, 46)
val:(27, 46)
y_train:(123,)
y_val:(27, 1)
[0]	validation_0-rmse:971.79233	validation_0-mae:844.04006
[1]	validation_0-rmse:741.80679	validation_0-mae:616.53053
[2]	validation_0-rmse:590.51839	validation_0-mae:459.64535
[3]	validation_0-rmse:482.86501	validation_0-mae:364.35498
[4]	validation_0-rmse:405.46137	validation_0-mae:291.21950
[5]	validation_0-rmse:356.93320	validation_0-mae:265.17385
[6]	validation_0-rmse:327.89818	validation_0-mae:248.62334
[7]	validation_0-rmse:315.76786	validation_0-mae:236.71356
[8]	validation_0-rmse:311.67944	validation_0-mae:232.02247
[9]	validation_0-rmse:307.30974	validation_0-mae:230.36590
[10]	validation_0-rmse:306.81361	validation_0-mae:230.02015
[11]	validation_0-rmse:303.29897	validation_0-mae:229.02983
[12]	validation_0-rmse:306.02268	validation_0-mae:230.24911
[13]	validation_0-rmse:304.06716	validation_0-mae:228.05274
[14]	validation_0-rmse:303.88006	validation_0-mae:226.29853
[15]	validation_0-rmse:303.30321	validation

...

LASSO------


2023/07/15 02:48:30 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\mlflow\models\signature.py:144: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2023/07/15 02:48:49 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\michi\Anaconda3\envs\e

Training lasso model done...
---EVALUATION AND LOGGING TO MLFLOW------ lasso


2023/07/15 02:48:51 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\mlflow\data\pandas_dataset.py:116: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2023/07/15 02:48:53 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\michi\Anaconda3\env

,model,mae,mse,r2,mae_test,mse_test,r2_test,mae_train,mse_train,r2_train
0,baseline-rent,285.556400,140135.418730,0.580289,237.832650,92491.552430,0.686215,NaN,NaN,NaN
1,xgb,224.410332,91992.834517,0.724478,119.411964,31594.955069,0.892812,41.488575,3255.965514,0.987830
2,ridge,223.964342,91430.988701,0.726161,190.765468,58641.051061,0.801055,139.655460,31126.799550,0.883657
3,rf,196.060289,86419.836355,0.741169,146.189547,34507.230467,0.882931,63.729311,8694.094695,0.967504
4,elasticnet,240.045151,98402.832902,0.705280,195.061738,55630.255891,0.811270,181.153074,52734.798647,0.802892
5,linear,258.883019,123548.054537,0.629969,200.569974,77504.414036,0.737060,134.096342,29350.289446,0.890297
6,lasso,227.731555,95994.975938,0.712491,194.851164,62413.160833,0.788258,139.543955,31260.965459,0.883155
